In [16]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
# from influxdb_connector import Influxdb
import pandas as pd


# Read csv file into pd dataFrame

In [20]:
csvfile = "DailyDelhiClimateMissing.csv"
df = pd.read_csv(csvfile, sep=",", engine='python',index_col=False)

display(df.head(5))

,date,meantemp,humidity,wind_speed,meanpressure
0,2017-01-01,15.913043,85.869565,2.743478,59.000000
1,2017-01-02,NaN,77.222222,NaN,1018.277778
2,2017-01-03,NaN,81.888889,4.016667,1018.333333
3,2017-01-04,18.700000,70.050000,NaN,NaN
4,2017-01-05,18.388889,74.944444,3.300000,1014.333333


# Convert date into linux timestamp. 

In [22]:
df['date'] = pd.to_datetime(df['date'])
print(df.columns)

Index(['date', 'meantemp', 'humidity', 'wind_speed', 'meanpressure'], dtype='object')


# Initialize InfluxDB

In [23]:
# arguments used for influxdb api
token = "aSccp32TgHYEOV0Ztt1ZeuyCHSs0-rlVBxmRTX69dsD5yO7_Bxj4CXnz4DSX0iKgzaTTaJC8jdH8iMaOEZy5Gw=="
org = "kris1.pan@intel.com"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"
bucket = "test"
measurement = "t3"
range_start = "-10y"
range_end = "now()"
time_col = "date"

client = InfluxDBClient(url=url, token=token , org=org)

# Write df into InfluxDB

In [24]:
with InfluxDBClient(url=url, token=token, org=org) as client:
    client.write_api(write_options=SYNCHRONOUS).write(bucket=bucket, record=df, data_frame_measurement_name=measurement, data_frame_timestamp_column=time_col)
    print('done')

done


# Query Pandas df from InfluxDB

In [25]:
query_api = influxdb_client.InfluxDBClient(url=url, token=token, org=org).query_api()
query = f"""from(bucket: "{bucket}")
 |> range(start: {range_start}, stop: {range_end}) 
 |> filter(fn: (r) => r._measurement == "{measurement}") 
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
"""
data = query_api.query_data_frame(query=query, org=org)

# remove meta data colums from the data
meta_columns = ['result','table','_start','_stop','_measurement']
data = data.drop(meta_columns, axis=1)
display(data)

,_time,humidity,meanpressure,meantemp,wind_speed
0,2017-01-01 00:00:00+00:00,85.869565,59.000000,15.913043,2.743478
1,2017-01-02 00:00:00+00:00,77.222222,1018.277778,NaN,NaN
2,2017-01-03 00:00:00+00:00,81.888889,1018.333333,NaN,4.016667
3,2017-01-04 00:00:00+00:00,70.050000,NaN,18.700000,NaN
4,2017-01-05 00:00:00+00:00,74.944444,1014.333333,18.388889,3.300000
